In [2]:
import sys
sys.path.append('../')

In [3]:
from Proximity import prox_model, Proximity_Indicator
from brix import Indicator, Handler, Grid_maker
import US_State
import datetime
import geopandas as gpd
import json
import requests

# In this example, we will:
    - download some census data filtered to the area surrounding the grid
    - create and deploy a proximity indicator

In [4]:
table_name='epa'
state_fips=6
geom_type='block'

# Set up a back end module

### Create the Handler

In [5]:
H=Handler(table_name)

https://cityscope-api.smartaraucania.org/api/tables/list/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epa/meta/hashes/ Attempt: 0


To intialise the proximity indicator, we need geodataframes of (i) the static places/POIs and (ii) the GEOGRID

In [6]:
geogrid=H.get_geogrid_data(include_geometries=True).as_df()

https://cityscope-api.smartaraucania.org/api/table/epa/GEOGRIDDATA/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epa/GEOGRID/ Attempt: 0


c:\Users\user1\anaconda3\envs\spacialModules\lib\site-packages\brix\classes.py:981: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  geos = gpd.GeoDataFrame(geos.drop('geometry',1),geometry=geos['geometry'].apply(lambda x: shape(x))) # no crs to avoid warning
c:\Users\user1\anaconda3\envs\spacialModules\lib\site-packages\brix\classes.py:251: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  geogrid_data = gpd.GeoDataFrame(geogrid_data.drop('geometry',1),geometry=geogrid_data['geometry'].apply(lambda x: shape(x)),crs='EPSG:4326')


### Get some baseline places data by downloading geometres and demographic data from census

In [7]:
lon, lat = geogrid.unary_union.centroid.x, geogrid.unary_union.centroid.y

In [8]:
# st=US_State.US_State(state_fips=state_fips, year=2019, geom_type=geom_type)
# st.get_geometry()
# st.get_lodes_data( include=['wac', 'rac'])
# st.add_lodes_cols_to_shape()

Subset the geometry by distance from the grid to reduce the size 

In [9]:
# st.subset_geom_by_distance(centre_x_y=[lon, lat], 
#                                    radius=5000, name='model_area')
# zones_subset=st.return_geometry('model_area')
# zones_subset.explore()

In [10]:
zones=gpd.read_file('../tables/epa/zones.geojson')

In [11]:
# zones.head()

In [12]:
zones.explore()

ImportError: The 'folium', 'matplotlib' and 'mapclassify' packages are required for 'explore()'. You can install them using 'conda install -c conda-forge folium matplotlib mapclassify' or 'pip install folium matplotlib mapclassify'.

### Define the proximity targets
- column is the column of the GeoDataFrame corresponding to this POI. The column can be binary (indicating whther the row is one these POIs) or numerical (indicating the capacity of the POI contained in this place.
- 'from' represents the relevant source location for this POI. eg. if we want a POI (eg. employment) to be located close to where people live, then 'from' should be the column representing residential population.
- 'max' is used in the linear normalisation of indicators. If the proximity result is equal to or greater than the max, the score will be 1.

In [ ]:
target_settings={'housing': {'column': 'res_total','max': 100000, 'from': 'emp_total'},
                 'healthcare': {'column': 'emp_naics_62','max': 10000, 'from': 'res_total'},
                 'jobs': {'column': 'emp_total','max': 100000, 'from': 'res_total'},
                'shopping': {'column': 'emp_naics_44-45','max': 10000, 'from': 'res_total'},
                'education': {'column': 'emp_naics_61','max': 10000, 'from': 'res_total'},
                'entertainment': {'column': 'emp_naics_71','max': 10000, 'from': 'res_total'}}

### Create the proximity indicator.
- 'max_dist' is the network search radius in meters. eg. for 15 minutes walking accessibility, use max_dist=1200 (assuming average espeed of 4.8km/hr)

In [ ]:
p=Proximity_Indicator(static_places=zones, geogrid=geogrid, 
                      max_dist=2400, target_settings=target_settings)

/Users/ronandoorley/Documents/Media_Lab/CityScope/CS_Spatial_Modules/examples/../Proximity.py:35: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geom_1['zone_area']=geom_1.geometry.area
/Users/ronandoorley/Documents/Media_Lab/CityScope/CS_Spatial_Modules/examples/../Proximity.py:37: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  all_intersect['intersect_area']=all_intersect.geometry.area


Getting network inside buffered outline from OSM


/Users/ronandoorley/Documents/Media_Lab/CityScope/CS_Spatial_Modules/examples/../Proximity.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  longitude=sum([c.x for c in geom.geometry.centroid])/len(geom)
/opt/anaconda3/envs/prox/lib/python3.10/site-packages/osmnet/load.py:445: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.
  gdf.crs = crs
/opt/anaconda3/envs/prox/lib/python3.10/site-packages/pyproj/crs/crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/opt/anaconda3/envs/prox/lib/

Requesting network data within bounding box from Overpass API in 1 request(s)
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]["foot"!~"no"]["pedestrians"!~"no"](37.42688986,-122.18364460,37.50397158,-122.08612927);>;);out;'}"
Downloaded 9,473.7KB from www.overpass-api.de in 3.27 seconds
Downloaded OSM network data within bounding box from Overpass API in 1 request(s) and 3.40 seconds


/opt/anaconda3/envs/prox/lib/python3.10/site-packages/osmnet/load.py:171: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  ' {:,.2f} seconds'.format(len(geometry), time.time()-start_time))


Returning OSM data with 57,758 nodes and 12,224 ways...
Edge node pairs completed. Took 4.87 seconds
Returning processed graph with 15,678 nodes and 21,930 edges...
Completed OSM data download and Pandana node and edge table creation in 8.77 seconds
Generating contraction hierarchies with 8 threads.
Setting CH node vector of size 15678
Setting CH edge vector of size 22236
Range graph removed 1242 edges of 44472
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%
Finding closest nodes to each place


/Users/ronandoorley/Documents/Media_Lab/CityScope/CS_Spatial_Modules/examples/../Proximity.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  longitude=sum([c.x for c in geom.geometry.centroid])/len(geom)


Building distance matrix
Finding closest nodes to each place


/Users/ronandoorley/Documents/Media_Lab/CityScope/CS_Spatial_Modules/examples/../Proximity.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  longitude=sum([c.x for c in geom.geometry.centroid])/len(geom)


In [ ]:
H.add_indicator(p)
H.listen()

/opt/anaconda3/envs/prox/lib/python3.10/site-packages/brix/classes.py:711: UserWarning: Indicator 0001 already exists and will be overwritten
  warn(f'Indicator {indicatorName} already exists and will be overwritten')


https://cityio.media.mit.edu/api/table/epa/GEOGRIDDATA/ Attempt: 0


/opt/anaconda3/envs/prox/lib/python3.10/site-packages/brix/classes.py:251: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  geogrid_data = gpd.GeoDataFrame(geogrid_data.drop('geometry',1),geometry=geogrid_data['geometry'].apply(lambda x: shape(x)),crs='EPSG:4326')


emp_naics_62
emp_naics_44-45
emp_naics_61
emp_naics_71
Proximity calculation took 0:00:01.026966 seconds


/Users/ronandoorley/Documents/Media_Lab/CityScope/CS_Spatial_Modules/examples/../Proximity.py:160: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  heatmap.geometry=heatmap.geometry.centroid


...

/opt/anaconda3/envs/prox/lib/python3.10/site-packages/brix/classes.py:251: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  geogrid_data = gpd.GeoDataFrame(geogrid_data.drop('geometry',1),geometry=geogrid_data['geometry'].apply(lambda x: shape(x)),crs='EPSG:4326')


emp_naics_62
emp_naics_44-45
emp_naics_61
emp_naics_71
Proximity calculation took 0:00:00.930954 seconds


/Users/ronandoorley/Documents/Media_Lab/CityScope/CS_Spatial_Modules/examples/../Proximity.py:160: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  heatmap.geometry=heatmap.geometry.centroid


Done with update
Local grid hash: BFTga8RwNze9Dz15pTni4JULqeScqgPmf8yrH4eoFwdh
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/ha

KeyboardInterrupt: 